## Bayes Nets

In this assignment, we'll implement several of the ideas from class, including inference by enumeration, variable elimination, and prior sampling.

To start, we'll need a data structure to store the conditional probability tables. We'll use a pandas [DataFrame](https://pythonbasics.org/pandas-dataframe/) to represent the table, which will make it easier when we start filtering and joining tables to do inference. You'll need to install pandas (`pip install pandas` or with conda).

In [1]:
import pandas as pd

class CPT:
    """
      A conditional probability table. We assume all variables are binary.
      Params:
        variables...list of variable names. We assume the final variable is the "output" variable (i.e., to the left of the conditional bar)
        table.......list of lists containing the variable assignments for each row, as well as the probability for that row.        
      E.g., for P(A|B), A is the output_variable, and B is the input_variables.
      >>> CPT(['B','A'], [[1,1,.9], [1,0,.1], [0,1,.6], [0,0,.4]])
      
      The table will be converted into a pandas DataFrame, with an additional column name appended called 'P' for the probability.
    """
    def __init__(self, variables, table):
        self.variables = variables
        self.table = pd.DataFrame(table, columns=variables + ['P'])
        # make sure variable assignments are ints.
        self.table = self.table.astype({v: int for v in variables})
    
    def _repr_html_(self):
        # for pretty printing
        return self.table._repr_html_()
        
    def filter_to_evidence(self, evidence):
        """
        Return a new CPT object where the rows have been 
        filtered to match the evidence.
        
        Params:
          evidence...dict from variable name to {0,1} value
        """
        df = self.table
        variable = self.variables
        for key in evidence.keys():
            value = evidence[key]
            if key in variable:
                df = df.loc[df[key] == value]   
        # Actually df is the dataframe we want
        # But we still want to create a new cpt object using this df
        # Similar to CPT(['B','A'], [[1,1,.9], [1,0,.1], [0,1,.6], [0,0,.4]])
        columnlist = list(df.columns.values[:-1])
        newtable = df.to_numpy()
        return CPT(columnlist, newtable)
        ###

    
CPT(['B','A'], [[1,1,.9], [1,0,.1], [0,1,.6], [0,0,.4]])

,B,A,P
0,1,1,0.9
1,1,0,0.1
2,0,1,0.6
3,0,0,0.4


Next, please complete the `filter_to_evidence` method above to create a new CPT that has been filtered to only contain rows that match the given evidence. See the example below. Note that this returns a **new** CPT object; it doesn't modify the pre-existing CPT.

In [2]:
CPT(['B','A'], [[1,1,.9], [1,0,.1], [0,1,.6], [0,0,.4]]).filter_to_evidence({'A': 1})

,B,A,P
0,1,1,0.9
1,0,1,0.6


Next, we'll implement the BayesNet class. This just contains a list of CPTs. We also store for convenience the set of unique variable names across all CPTs.

In [3]:
class BayesNet:
    
    def __init__(self, cpts):
        self.cpts = cpts
        self.variables = [cpt.variables[-1] for cpt in cpts]
        
    def joint(self, assignment):
        """
        Return the joint probability of the given full variable assignment according to this BayesNet.
        """
        cpt_list = self.cpts # Each element is a cpt object
        l = len(cpt_list)    # The number of the cpts
        p = 1                # The initial probability
        for i in range(l):
            p = p * cpt_list[i].filter_to_evidence(assignment).table.P.iloc[0]
         
        return p
        pass
        ###
            
    def filter_to_evidence(self, evidence):
        return BayesNet([cpt.filter_to_evidence(evidence) for cpt in self.cpts])
    
    def _repr_html_(self):
        # for pretty printing
        return '\n'.join(c._repr_html_() for c in self.cpts)
        
        
# here we create the alarm network example from class.
def make_net():
    return BayesNet(
    [
        CPT(['Burglary'], [[1, .001], [0, .999]]),
        CPT(['Earthquake'], [[1, .002], [0, .998]]),
        CPT(['Burglary', 'Earthquake', 'Alarm'], 
           [
               [1,1,1,.95],
               [1,1,0,.05],
               [1,0,1,.94],
               [1,0,0,.06],
               [0,1,1,.29],
               [0,1,0,.71],
               [0,0,1,.001],
               [0,0,0,.999]       
           ]),
        CPT(['Alarm', 'JohnCalls'], [[1,1,.9], [1,0,.1], [0,1,.05], [0,0,.95]]),
        CPT(['Alarm', 'MaryCalls'], [[1,1,.7], [1,0,.3], [0,1,.01], [0,0,.99]]),
    ])

bn = make_net()
display(bn)

Implement the `joint` method in the `BayesNet` class above to compute the joint probability of a full variable assignment according to this Bayesian network. Example below.

In [4]:
bn.joint({'Burglary': 1, 'Earthquake': 0, 'Alarm': 1, 'JohnCalls': 0, 'MaryCalls': 1})

6.56684e-05

## Inference by enumeration

Now, let's implement inference using enumeration. Recall the steps to compute P(Q|E), potentially with hidden variables H.

1. Filter the CPTs to match the evidence E
2. Repeatedly join CPTs until only one remains.
3. Sum out H
4. Normalize over Q

We'll use as our running example P(Burglary | JohnCalls=1, MaryCalls=1)

**Step 1. Filter the CPTs to match the evidence E**

The first step is already completed for you via the `BayesNet.filter_to_evidence` method, which just calls the CPT.filter_to_evidence method for all CPTs in the network. Note that this returns a new network.

In [5]:
bn_filtered = bn.filter_to_evidence({'JohnCalls': 1, 'MaryCalls': 1})
display(bn_filtered)

**Step 2. Repeatedly join CPTs until only one remains.**

To do this, let's first implement a method that joins to CPTs.

In [6]:
def join(cpt1, cpt2):
    """
    Return a new factor (CPT) that is the result of joining cpt1 and cpt2.
    See the pandas merge method (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html)
    You can first merge the two DataFrames, then create a new column P that is the product of the probabilities from 
    the two original tables.
    Note that the result is not technically a CPT in the way we've used it above. That is, in the table, the P does 
    not represent P(last variable| other variables). You should think of the CPT class as representing both
    CPTs and general factors.
    """
    df1 = cpt1.table
    var1 = cpt1.variables
    df2 = cpt2.table
    var2 = cpt2.variables
    intersection = [i for i in var1 if i in var2]
    # union = list(set(var1).union(set(var2)))
    # if intercet is empty list or not
    if intersection:
        new1 = df1.merge(df2,on=intersection)
        new_p = new1.P_x * new1.P_y

        new2 = new1.drop(columns=['P_x', 'P_y'])
        new2['P'] = new_p
        # Actually new2 is the dataframe we want
        # But we still want to create a new cpt object using this new2
        # Similar to CPT(['B','A'], [[1,1,.9], [1,0,.1], [0,1,.6], [0,0,.4]])
        columnlist = list(new2.columns.values[:-1])
        newtable = new2.to_numpy()
        return CPT(columnlist, newtable)
    else:
        return None

    pass
    ###

print('joining')
display(bn.cpts[1])
print('and')
display(bn.cpts[2])
print('yields')
join(bn.cpts[1], bn.cpts[2])

joining


,Earthquake,P
0,1,0.002
1,0,0.998


and


,Burglary,Earthquake,Alarm,P
0,1,1,1,0.950
1,1,1,0,0.050
2,1,0,1,0.940
3,1,0,0,0.060
4,0,1,1,0.290
5,0,1,0,0.710
6,0,0,1,0.001
7,0,0,0,0.999


yields


,Earthquake,Burglary,Alarm,P
0,1,1,1,0.001900
1,1,1,0,0.000100
2,1,0,1,0.000580
3,1,0,0,0.001420
4,0,1,1,0.938120
5,0,1,0,0.059880
6,0,0,1,0.000998
7,0,0,0,0.997002


In [7]:
# if we try to merge two CPTs with no variables in common, just return None
display(bn.cpts[0])
display(bn.cpts[-1])
print(join(bn.cpts[0], bn.cpts[-1]))

,Burglary,P
0,1,0.001
1,0,0.999


,Alarm,MaryCalls,P
0,1,1,0.70
1,1,0,0.30
2,0,1,0.01
3,0,0,0.99


None


Now, we need a method to keep joining tables until only one remains. One way to do this is to start with the full list of CPTs from the bayes net. Then, loop over each variable, find all CPTs that contain it, remove them from the cpt list, merge them together, then add them back to the CPT list.

In [8]:
def join_all(bn):
    """
    Join all CPTs in the Bayes Net. Reaturn a single factor (CPT) containing all variables.
    """
    cpt_list = bn.cpts
    variable_list = bn.variables
    # loop over each variable
    for var in variable_list:
        cpt_contain_var = list()
        # find all CPTs that contain that variable
        for cpt in cpt_list:
            if var in cpt.variables:
                # remove them from the cpt list
                # add them to a new list
                cpt_contain_var.append(cpt)
                cpt_list.remove(cpt)
        merge = cpt_contain_var.pop(0)
        for left in cpt_contain_var:
            merge = join(merge, left)
        cpt_list.append(merge)
    # At last, we need to join all item in cpt_list
    join_all = cpt_list.pop(0)
    for left_join in cpt_list:
        join_all = join(join_all,left_join)

    return join_all


    pass
    ###

joined_cpt = join_all(bn_filtered)
display(joined_cpt)

,Alarm,JohnCalls,Earthquake,Burglary,MaryCalls,P
0,1,1,1,1,1,1.197000e-06
1,1,1,1,0,1,3.650346e-04
2,1,1,0,1,1,5.910156e-04
3,1,1,0,0,1,6.281113e-04
4,0,1,1,1,1,5.000000e-11
5,0,1,1,0,1,7.092900e-07
6,0,1,0,1,1,2.994000e-08
7,0,1,0,0,1,4.980025e-04


**Step 3. Sum out H**

Given the joined factor, we next need to sum out any hidden variables.

In [9]:
def sumout(cpt, query_variable, hidden_variables):
    """
    Return a new CPT formed by summing out the hidden variables 
    for this cpt. We assume there is a single query variable, 
    but there may be multiple hidden_variables.
    """
    df = cpt.table
    variable_list = cpt.variables
    col = list(set(variable_list) - set(hidden_variables))
    df1 = df.drop(columns=hidden_variables)
    df2 = df1.groupby(by=col,as_index=False).sum()
    columnlist = list(df2.columns.values[:-1])
    newtable = df2.to_numpy()
    return CPT(columnlist, newtable)
    ###

print('summing MaryCalls from')
display(bn.cpts[-1])
print('yields')
sumout(bn.cpts[-1], 'Alarm', ['MaryCalls'])    

summing MaryCalls from


,Alarm,MaryCalls,P
0,1,1,0.70
1,1,0,0.30
2,0,1,0.01
3,0,0,0.99


yields


,Alarm,P
0,0,1.0
1,1,1.0


In [10]:
display(joined_cpt)
unnormalized_answer = sumout(joined_cpt, 'Burglary', ['Alarm', 'Earthquake'])
display(unnormalized_answer)

,Alarm,JohnCalls,Earthquake,Burglary,MaryCalls,P
0,1,1,1,1,1,1.197000e-06
1,1,1,1,0,1,3.650346e-04
2,1,1,0,1,1,5.910156e-04
3,1,1,0,0,1,6.281113e-04
4,0,1,1,1,1,5.000000e-11
5,0,1,1,0,1,7.092900e-07
6,0,1,0,1,1,2.994000e-08
7,0,1,0,0,1,4.980025e-04


,MaryCalls,JohnCalls,Burglary,P
0,1,1,0,0.001492
1,1,1,1,0.000592


**Step 4. Normalize**

Finally, we just need to normalize the `unnormalized_answer` CPT so the probabilities sum to one.

In [11]:
def normalize(cpt):
    """
    Return a new CPT that is normalized.
    """
    df = cpt.table
    z = df.P.sum()
    df['P'] = df['P'].div(z)
    columnlist = list(df.columns.values[:-1])
    newtable = df.to_numpy()
    return CPT(columnlist, newtable)
    ###
    
print('normalizing')
display(unnormalized_answer)
answer = normalize(unnormalized_answer)
print('yields')
display(answer)

normalizing


,MaryCalls,JohnCalls,Burglary,P
0,1,1,0,0.001492
1,1,1,1,0.000592


yields


,MaryCalls,JohnCalls,Burglary,P
0,1,1,0,0.715828
1,1,1,1,0.284172


We can put these four steps together into a single function.

In [12]:
def infer_by_enumeration(bn, query, hidden, evidence):
    """
    1. Filter CPTs to match evidence.
    2. Join all CPTs into factor over all variables.
    3. Sum out hidden variables
    4. Normalize
    
    For future comparisons, I'm also printing out the joined_cpt
    so we can see how big this gets.
    """
    bn_filtered = bn.filter_to_evidence(evidence)
    joined_cpt = join_all(bn_filtered)
    print('max cpt size: %d' % len(joined_cpt.table))
    print('max cpt')
    display(joined_cpt)
    unnormalized_answer = sumout(joined_cpt, query, hidden)
    answer = normalize(unnormalized_answer)
    return answer
    
# P(B|J=1,M=1)
infer_by_enumeration(make_net(), 'Burglary', ['Alarm', 'Earthquake'], {'JohnCalls': 1, 'MaryCalls': 1})

max cpt size: 8
max cpt


,Alarm,JohnCalls,Earthquake,Burglary,MaryCalls,P
0,1,1,1,1,1,1.197000e-06
1,1,1,1,0,1,3.650346e-04
2,1,1,0,1,1,5.910156e-04
3,1,1,0,0,1,6.281113e-04
4,0,1,1,1,1,5.000000e-11
5,0,1,1,0,1,7.092900e-07
6,0,1,0,1,1,2.994000e-08
7,0,1,0,0,1,4.980025e-04


,MaryCalls,JohnCalls,Burglary,P
0,1,1,0,0.715828
1,1,1,1,0.284172


In [13]:
# P(B|J=0,M=0)
infer_by_enumeration(make_net(), 'Burglary', ['Alarm', 'Earthquake'], {'JohnCalls': 0, 'MaryCalls': 0})

max cpt size: 8
max cpt


,Alarm,JohnCalls,Earthquake,Burglary,MaryCalls,P
0,1,0,1,1,0,5.700000e-08
1,1,0,1,0,0,1.738260e-05
2,1,0,0,1,0,2.814360e-05
3,1,0,0,0,0,2.991006e-05
4,0,0,1,1,0,9.405000e-08
5,0,0,1,0,0,1.334174e-03
6,0,0,0,1,0,5.631714e-05
7,0,0,0,0,0,9.367427e-01


,MaryCalls,JohnCalls,Burglary,P
0,0,0,0,0.99991
1,0,0,1,0.00009


In [14]:
# P(B|J=0,M=1)
infer_by_enumeration(make_net(), 'Burglary', ['Alarm', 'Earthquake'], {'JohnCalls': 0, 'MaryCalls': 1})

max cpt size: 8
max cpt


,Alarm,JohnCalls,Earthquake,Burglary,MaryCalls,P
0,1,0,1,1,1,1.330000e-07
1,1,0,1,0,1,4.055940e-05
2,1,0,0,1,1,6.566840e-05
3,1,0,0,0,1,6.979014e-05
4,0,0,1,1,1,9.500000e-10
5,0,0,1,0,1,1.347651e-05
6,0,0,0,1,1,5.688600e-07
7,0,0,0,0,1,9.462047e-03


,MaryCalls,JohnCalls,Burglary,P
0,1,0,0,0.993124
1,1,0,1,0.006876


In [15]:
# P(B|J=1,M=0)
infer_by_enumeration(make_net(), 'Burglary', ['Alarm', 'Earthquake'], {'JohnCalls': 1, 'MaryCalls': 0})

max cpt size: 8
max cpt


,Alarm,JohnCalls,Earthquake,Burglary,MaryCalls,P
0,1,1,1,1,0,5.130000e-07
1,1,1,1,0,0,1.564434e-04
2,1,1,0,1,0,2.532924e-04
3,1,1,0,0,0,2.691905e-04
4,0,1,1,1,0,4.950000e-09
5,0,1,1,0,0,7.021971e-05
6,0,1,0,1,0,2.964060e-06
7,0,1,0,0,0,4.930225e-02


,MaryCalls,JohnCalls,Burglary,P
0,0,1,0,0.99487
1,0,1,1,0.00513


## Inference by variable elimination

Next, let's implement variable elimination. You should be able to reuse some of the methods above, but the interleaving of joining/eliminating variables will differ. The final answers should be the same as above.

In [16]:
def infer_by_ve(bn, query, hidden, evidence):
    """
    1. Filter CPTs to match evidence.
    2. Repeat:
        + Pick a hidden variable
        + Join all factors with that variable
        + Sum out hidden variables
    3. Join all remaining factos
    4. Normalize
    
    For comparison, you should also print out the biggest CPT encountered
    as part of inference, as well as the number of rows it has (see example below).
    """
    # Filter CPTs to match evidence
    # Find the cpt reamin in the bn_filtered
    bn_filtered = bn.filter_to_evidence(evidence)
    cpt_in_filtered = [cpt for cpt in bn_filtered.cpts]
    max_cpt = None
    # Repeat
    for hidden_variable in hidden:
        cpts1 = []
        cpts2 = []
        for cpt in cpt_in_filtered:
            if hidden_variable in cpt.variables:
                cpts1.append(cpt)
            else:
                cpts2.append(cpt)
        # Join all factors with that variable 'hidden_variable'
        joined_cpt = cpts1[0]
        len_cpts1 = len(cpts1)
        for i in range(1,len_cpts1):
            joined_cpt = join(joined_cpt,cpts1[i])
        if max_cpt is None or len(max_cpt.table) < len(joined_cpt.table):
            max_cpt = joined_cpt
            
        # Sum out hidden variables 
        joined_cpt = sumout(joined_cpt,query,[hidden_variable])
        cpt_in_filtered = cpts2 + [joined_cpt]
    # Join all remaining factos
    joined_cpt = cpt_in_filtered[0]
    for i in range(1,len(cpt_in_filtered)):
        joined_cpt = join(joined_cpt,cpt_in_filtered[i])
    
    
    print('max cpt size: %d' % len(max_cpt.table))
    print('max cpt:')
    display(max_cpt)
    result = normalize(joined_cpt)
    return result
    ###

infer_by_ve(make_net(), 'Burglary', ['Alarm', 'Earthquake'], {'JohnCalls': 1, 'MaryCalls': 1})

max cpt size: 8
max cpt:


,Burglary,Earthquake,Alarm,JohnCalls,MaryCalls,P
0,1,1,1,1,1,0.598500
1,1,0,1,1,1,0.592200
2,0,1,1,1,1,0.182700
3,0,0,1,1,1,0.000630
4,1,1,0,1,1,0.000025
5,1,0,0,1,1,0.000030
6,0,1,0,1,1,0.000355
7,0,0,0,1,1,0.000500


,Burglary,JohnCalls,MaryCalls,P
0,1,1,1,0.284172
1,0,1,1,0.715828


In [17]:
infer_by_ve(make_net(), 'Burglary', ['Alarm', 'Earthquake'], {'JohnCalls': 0, 'MaryCalls': 0})

max cpt size: 8
max cpt:


,Burglary,Earthquake,Alarm,JohnCalls,MaryCalls,P
0,1,1,1,0,0,0.028500
1,1,0,1,0,0,0.028200
2,0,1,1,0,0,0.008700
3,0,0,1,0,0,0.000030
4,1,1,0,0,0,0.047025
5,1,0,0,0,0,0.056430
6,0,1,0,0,0,0.667755
7,0,0,0,0,0,0.939559


,Burglary,JohnCalls,MaryCalls,P
0,1,0,0,0.00009
1,0,0,0,0.99991


In [18]:
# P(B|J=0,M=1)
infer_by_ve(make_net(), 'Burglary', ['Alarm', 'Earthquake'], {'JohnCalls': 0, 'MaryCalls': 1})

max cpt size: 8
max cpt:


,Burglary,Earthquake,Alarm,JohnCalls,MaryCalls,P
0,1,1,1,0,1,0.066500
1,1,0,1,0,1,0.065800
2,0,1,1,0,1,0.020300
3,0,0,1,0,1,0.000070
4,1,1,0,0,1,0.000475
5,1,0,0,0,1,0.000570
6,0,1,0,0,1,0.006745
7,0,0,0,0,1,0.009490


,Burglary,JohnCalls,MaryCalls,P
0,1,0,1,0.006876
1,0,0,1,0.993124


In [19]:
# P(B|J=0,M=1)
infer_by_ve(make_net(), 'Burglary', ['Alarm', 'Earthquake'], {'JohnCalls': 1, 'MaryCalls': 0})

max cpt size: 8
max cpt:


,Burglary,Earthquake,Alarm,JohnCalls,MaryCalls,P
0,1,1,1,1,0,0.256500
1,1,0,1,1,0,0.253800
2,0,1,1,1,0,0.078300
3,0,0,1,1,0,0.000270
4,1,1,0,1,0,0.002475
5,1,0,0,1,0,0.002970
6,0,1,0,1,0,0.035145
7,0,0,0,1,0,0.049451


,Burglary,JohnCalls,MaryCalls,P
0,1,1,0,0.00513
1,0,1,0,0.99487


The max CPT is the same for enumeration and variable elimination. Construct a new bayes net and a corresponding query where the max CPT for variable elimination is smaller than that for enumeration.

In [20]:
def make_other_net():
    ### TODO
    return BayesNet(
    [
        CPT(['Query'], [[1, .001], [0, .999]]),
        CPT(['Hidden0'], [[1, .002], [0, .998]]),
        CPT(['Query', 'Hidden0', 'Hidden1'], 
           [
               [1,1,1,.95],
               [1,1,0,.05],
               [1,0,1,.94],
               [1,0,0,.06],
               [0,1,1,.29],
               [0,1,0,.71],
               [0,0,1,.001],
               [0,0,0,.999]       
           ]),
        CPT(['Hidden1', 'Hidden2'], [[1,1,.9], [1,0,.1], [0,1,.05], [0,0,.95]]),
        CPT(['Hidden2', 'Evidence'], [[1,1,.7], [1,0,.3], [0,1,.01], [0,0,.99]]),
    ])   
    ###
    
# make the net
# infer_by_enumeration
# infer_by_ve

## Sampling
Finally, we'll look briefly at prior sampling. First, we'll define a network with less extreme CPT values so prior sampling has a chance to be useful.

In [21]:
def make_net2():
    return BayesNet(
    [
        CPT(['Burglary'], [[1, .1], [0, .9]]),
        CPT(['Earthquake'], [[1, .2], [0, .8]]),
        CPT(['Burglary', 'Earthquake', 'Alarm'], 
           [
               [1,1,1,.95],
               [1,1,0,.05],
               [1,0,1,.94],
               [1,0,0,.06],
               [0,1,1,.29],
               [0,1,0,.71],
               [0,0,1,.1],
               [0,0,0,.9]       
           ]),
        CPT(['Alarm', 'JohnCalls'], [[1,1,.9], [1,0,.1], [0,1,.25], [0,0,.75]]),
        CPT(['Alarm', 'MaryCalls'], [[1,1,.7], [1,0,.3], [0,1,.1], [0,0,.9]]),
    ])

bn2 = make_net2()
bn2

Below, I've given an example of sampling from a single CPT that has been filtered to evidence.
See https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html

Here, we are again assuming that the final variable in the table is the one that we are sampling over (so, `JohnCalls` in the example below). This is why the magical `-2` is in the `sample` function.

Note that I'm not using any random seed initializer, so this will produce different results each time it is run. Thus, your results are unlikely to match mine exactly for this section.

In [22]:
# Counter is helpful for...counting.
from collections import Counter

def sample(cpt):
    return int(cpt.table.sample(weights=cpt.table.P).values[0][-2])

cpt = bn.cpts[3]
cpt = cpt.filter_to_evidence({'Alarm': 1})
print('drawing 100 samples from')
display(cpt)

Counter(sample(cpt) for _ in range(100))

drawing 100 samples from


,Alarm,JohnCalls,P
0,1,1,0.9
1,1,0,0.1


Counter({1: 92, 0: 8})

Next, let's implement prior sampling. We'll just assume that the order of the `cpts` variable in the `BayesNet` is the order in which we will do the sampling. Generally, we'd have to find the topological sort of the network.

In [23]:
def prior_sampling(bn):
    """
    Create a single sample of an assignment to all values in this network using prior sampling.
    Iterate through each CPT in order, sample a variable assignment, and fix it as evidence for future
    samples.
    
    Returns:
     a tuple of {0,1} values corresponding to the variable assignment
     the variable order is determined by the cpt order in the bayes net.
    """
    ### TODO
    evidence = {}
    for var in bn.variables:
        for cpt in bn.cpts:
            evidence[var] = sample(cpt.filter_to_evidence(evidence))
    return tuple(evidence[v] for v in bn.variables)
    ###
prior_sampling(bn2)

(0, 0, 0, 0, 0)

I've implemented below a method that calls your prior sampling method many times and compiles the counts into a DataFrame.

In [24]:
def do_prior_sampling(bn, n_samples=100):
    cts = Counter()
    for _ in range(n_samples):
        cts.update([prior_sampling(bn2)])
    variables = [cpt.variables[-1] for cpt in bn.cpts]
    return pd.DataFrame([list(i) + [j] for i,j in cts.items()], columns = variables + ['n'])
    
samples = do_prior_sampling(bn2, 100)
samples

,Burglary,Earthquake,Alarm,JohnCalls,MaryCalls,n
0,0,1,0,0,0,14
1,1,1,1,0,0,2
2,0,1,1,1,1,12
3,0,1,1,0,0,7
4,0,0,0,1,1,2
5,1,1,0,0,0,4
6,1,0,0,0,0,12
7,1,1,1,1,1,10
8,0,0,0,0,0,24
9,1,0,1,1,1,2


Finally, let's use this sample to perform approximate inference.

In [25]:
def empirical_estimate_joint(samples, evidence):
    """
    Return the empirical probability of the assignment to all variables given by evidence. P(E).
    This is simply the fraction of all samples that match this evidence.
    """
    ### TODO
    masks = [samples[variable] == value for variable,value in evidence.items()]
    count = samples['n'][sum(masks)==len(masks)]
    if len(count) > 0:
        count = count.iloc[0]
    else:
        count = 0
    return count/samples['n'].sum()
    ## #
    
empirical_estimate_joint(samples, {'Burglary': 0, 'Earthquake': 1, 'Alarm': 0, 'JohnCalls': 0, 'MaryCalls': 0})

0.14

In [26]:
# how does this compare to the "true" joint probability?
bn2.joint({'Burglary': 0, 'Earthquake': 1, 'Alarm': 0, 'JohnCalls': 0, 'MaryCalls': 0})

0.086265

In [27]:
# how does the quality of this estimate change as I increase my sample size.
evidence = {'Burglary': 0, 'Earthquake': 1, 'Alarm': 0, 'JohnCalls': 0, 'MaryCalls': 0}
for n in [50,100,200,500]:
    print(n, empirical_estimate_joint(do_prior_sampling(bn2, n), evidence))

50 0.04
100 0.06
200 0.09
500 0.102


Finally, finally, let's do this also when there are query and hidden variables.

In [28]:
def empirical_estimate_conditional(samples, query, hidden, evidence):
    """
    Estimate P(Q|H,E) from the sample.
    To do so, we first filter based on the evidence, sumout the hidden variables, then normalize.
    """
    # Filter 
    masks = [samples[variable]==value for variable,value in evidence.items()]
    filtered = samples.loc[sum(masks)==len(masks)]
    
    # Sum out hidden variables
    agg = filtered.drop(columns=hidden)
    vars = list(agg.columns[:-1])
    agg = agg.groupby(vars,as_index=False).sum()
    
    # Normalize
    p_table = agg.rename(columns={'n':'P'})
    p_table['P'] /= p_table['P'].sum()
    return p_table
   
    
    ###
    
empirical_estimate_conditional(samples, 'Earthquake', ['Alarm', 'Earthquake'], {'JohnCalls': 0, 'MaryCalls': 0})

,Burglary,JohnCalls,MaryCalls,P
0,0,0,0,0.686567
1,1,0,0,0.313433


In [29]:
# again, we can compare this to the exact approach.
infer_by_ve(bn2, 'Earthquake', ['Alarm', 'Earthquake'], {'JohnCalls': 0, 'MaryCalls': 0})

max cpt size: 8
max cpt:


,Burglary,Earthquake,Alarm,JohnCalls,MaryCalls,P
0,1,1,1,0,0,0.02850
1,1,0,1,0,0,0.02820
2,0,1,1,0,0,0.00870
3,0,0,1,0,0,0.00300
4,1,1,0,0,0,0.03375
5,1,0,0,0,0,0.04050
6,0,1,0,0,0,0.47925
7,0,0,0,0,0,0.60750


,Burglary,JohnCalls,MaryCalls,P
0,1,0,0,0.01262
1,0,0,0,0.98738


Congrats, you made it to the end! Hopefully you now have a deeper understanding of inference in BayesNets.